In [1]:
print('merge_notifica')
from sys import exit
import numpy as np
import pandas as pd
from os.path import dirname, join, isfile, isdir
from os import makedirs
from datetime import datetime, timedelta, date

from bulletin.data.notifica import Notifica
from bulletin.data.casos_confirmados import CasosConfirmados
from bulletin.commom.utils import Timer, get_better_notifica, auto_fit_columns
from bulletin.commom.static import meses
from bulletin.metabase.request import download_metabase

pd.set_option('display.max_columns', None)

output = join("output","correcoes","merge_notifica")

if not isdir(output):
    makedirs(output)

timerAll = Timer()
timer = Timer()

merge_notifica


In [2]:
cc = CasosConfirmados()
cc.load()
print(cc.shape())

notifica = Notifica()
notifica.load()

print(notifica.shape())

(661109, 11982, 472175, 176952)
(2562547, 1164225, 12911, 1355566, 2334)


In [3]:
casosc = cc.get_casos()
casosn = notifica.get_casos()
# casosn = casosn.loc[(casosn['duplicado']==0) | ((casosn['duplicado']==1) & (casosn['manter']==1))]
print(casosn.shape)

(2562547, 41)


In [4]:
idx_casos_confirmados = casosc.loc[casosc['hash_resid'].isin(casosn['hash_resid'])].index.tolist()
idx_casos_confirmados += casosc.loc[casosc['hash_atend'].isin(casosn['hash_atend'])].index.tolist()
idx_casos_confirmados += casosc.loc[casosc['hash_diag'].isin(casosn['hash_diag'])].index.tolist()

idx_casos_confirmados = set(idx_casos_confirmados)
casos_confirmados_nao_notifica = casosc.loc[ set(casosc.index.tolist()) - idx_casos_confirmados ].sort_values('ordem')
casos_confirmados_nao_notifica.shape

(27551, 25)

In [5]:
idx_casos_notifica = casosn.loc[casosn['hash_resid'].isin(casosc['hash_resid'])].index.tolist()
idx_casos_notifica += casosn.loc[casosn['hash_atend'].isin(casosc['hash_atend'])].index.tolist()
idx_casos_notifica += casosn.loc[casosn['hash_diag'].isin(casosc['hash_diag'])].index.tolist()

idx_casos_notifica = set(idx_casos_notifica)
casos_confirmados_notifica = casosn.loc[ idx_casos_notifica ].sort_values('id')
casos_confirmados_notifica.shape

(744311, 41)

In [6]:
#merge hash_resid
casos_hash_resid = casosc.loc[casosc['hash_resid'].isin(casosn['hash_resid'])]
print(f"casos_hash_resid {len(casos_hash_resid)}")
notifica_hash_resid = casosn.loc[(casosn['hash_resid'].isin(casosc['hash_resid']))]
print(f"notifica_hash_resid {len(notifica_hash_resid)}")
merge_hash_resid = pd.merge(casos_hash_resid,notifica_hash_resid[['hash_resid','id']])
print(f"merge_hash_resid {len(merge_hash_resid)}")
merge_hash_resid = merge_hash_resid.drop_duplicates('ordem')
print(f"merge_hash_resid {len(merge_hash_resid)}\n")

#merge hash_atend
casos_hash_atend = casosc.loc[casosc['hash_atend'].isin(casosn['hash_atend'])]
print(f"casos_hash_atend {len(casos_hash_atend)}")
notifica_hash_atend = casosn.loc[(casosn['hash_atend'].isin(casosc['hash_atend']))]
print(f"notifica_hash_atend {len(notifica_hash_atend)}")
merge_hash_atend = pd.merge(casos_hash_atend,notifica_hash_atend[['hash_atend','id']])
print(f"merge_hash_atend {len(merge_hash_atend)}")
merge_hash_atend = merge_hash_atend.drop_duplicates('ordem')
print(f"merge_hash_atend {len(merge_hash_atend)}\n")

#merge hash_diag
casos_hash_diag = casosc.loc[casosc['hash_diag'].isin(casosn['hash_diag'])]
print(f"casos_hash_diag {len(casos_hash_diag)}")
notifica_hash_diag = casosn.loc[(casosn['hash_diag'].isin(casosc['hash_diag']))]
print(f"notifica_hash_diag {len(notifica_hash_diag)}")
merge_hash_diag = pd.merge(casos_hash_diag,notifica_hash_diag[['hash_diag','id']])
print(f"merge_hash_diag {len(merge_hash_diag)}")
merge_hash_diag = merge_hash_diag.drop_duplicates('ordem')
print(f"merge_hash_diag {len(merge_hash_diag)}\n")

#trasnforma essa lista de index em um conjunto, para deixar apenas os index unicos
idx_casos_confirmados = set(casos_hash_resid.index.tolist() + casos_hash_atend.index.tolist() + casos_hash_diag.index.tolist())
print(f"idx_casos_confirmados {len(idx_casos_confirmados)}")

#realiza a diferença de conjunto para pegar os index dos que não foram encotrados no notifica
casos_confirmados_nao_notifica = casosc.loc[ set(casosc.index.tolist()) - idx_casos_confirmados ].sort_values('ordem')
print(f"casos_confirmados_nao_notifica {len(casos_confirmados_nao_notifica)}") ## aqui o numero total é 27910 casos não encontrados no notifica
len(idx_casos_confirmados) + len(casos_confirmados_nao_notifica)

casos_hash_resid 610212
notifica_hash_resid 722209
merge_hash_resid 722209
merge_hash_resid 610212

casos_hash_atend 577764
notifica_hash_atend 664517
merge_hash_atend 671334
merge_hash_atend 577764

casos_hash_diag 554976
notifica_hash_diag 558270
merge_hash_diag 564842
merge_hash_diag 554976

idx_casos_confirmados 633558
casos_confirmados_nao_notifica 27551


661109

In [7]:
merge_casos_notifica = pd.concat([merge_hash_resid,merge_hash_atend,merge_hash_diag,casos_confirmados_nao_notifica]).sort_values('ordem')
ordem_duplicados = merge_casos_notifica.loc[merge_casos_notifica.duplicated('ordem', keep=False)]
merge_casos_notifica = pd.concat([merge_hash_resid,merge_hash_atend,merge_hash_diag,casos_confirmados_nao_notifica]).drop_duplicates('ordem')

print(len(merge_casos_notifica))

661109


In [8]:
casos_merged = merge_casos_notifica.copy()
casos_merged['id'] = casos_merged['id'].fillna('-1').astype(int)

casos_merged.loc[casos_merged['recuperado']==1,'evolucao'] = 1
casos_merged.loc[casos_merged['obito']==1,'evolucao'] = 2
casos_merged.loc[casos_merged['ativo']==1,'evolucao'] = 3

casos_merged = casos_merged.rename(columns={
    'identificacao': 'id_access',
    'laboratorio': 'exame',
    'data_obito': 'data_evolucao',
    'data_com_obito': 'data_com_evolucao'
    })

casos_merged.loc[(casos_merged['data_com_evolucao'].isnull() & (casos_merged['evolucao']==2)),'data_com_evolucao'] = casos_merged.loc[(casos_merged['data_com_evolucao'].isnull() & (casos_merged['evolucao']==2)),'data_evolucao']

casos_merged = casos_merged[['id_access','id','paciente','sexo','idade','rs','ibge_resid','mun_resid','data_1o_sintomas','ibge_atend','mun_atend','exame','data_diagnostico','data_comunicacao','evolucao','data_evolucao','data_com_evolucao']].sort_values('id_access')

obitos_merged = casos_merged.loc[casos_merged['evolucao']==2,['id_access','id','paciente','sexo','idade','rs','ibge_resid','mun_resid','data_diagnostico','data_evolucao','data_com_evolucao']].sort_values('data_com_evolucao')

casos_municipios = casos_merged.groupby(['rs','ibge_resid','evolucao'])[['id']].count().unstack(level=[2]).droplevel(0,1).fillna(0).astype(int).rename(columns={1.0:'recuperados',2.0:'obitos',3.0:'ativos'})
casos_municipios['casos'] = casos_municipios.sum(axis=1)
casos_municipios = casos_municipios[['casos','obitos','recuperados','ativos']]
#totais

casos_regionais = casos_municipios.groupby('rs')[['casos','obitos','recuperados','ativos']].sum()

total_pr = casos_regionais.loc[casos_regionais.index!=99].sum(axis=0)
total_geral = casos_regionais.sum(axis=0)

casos_municipios = casos_municipios.droplevel(0,0)

casos_municipios.loc[('total_pr')] = total_pr           #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
casos_municipios.loc['total_geral'] = total_geral 


casos_regionais.loc['total_pr'] = total_pr
casos_regionais.loc['total_geral'] = total_geral

print(casos_merged.shape)
print(obitos_merged.shape)

(661109, 17)
(11982, 11)


In [9]:
casos_municipios

evolucao,casos,obitos,recuperados,ativos
ibge_resid,,,,
4101200,1543,38,1379,126
4109500,270,5,221,44
4109609,2959,69,2270,620
4115705,1718,39,1426,253
4116208,1301,29,1170,102
...,...,...,...,...
4123857,253,8,213,32
4125001,489,9,455,25
9999999,4699,94,1056,3549


In [10]:
casos_regionais

evolucao,casos,obitos,recuperados,ativos
rs,,,,
1,21734,397,13142,8195
2,187095,4705,145310,37080
3,36385,616,24183,11586
4,7151,107,5927,1117
5,18226,233,11159,6834
6,4762,65,3400,1297
7,16507,234,13166,3107
8,23705,264,18484,4957
9,40294,572,29939,9783


In [11]:
writer = pd.ExcelWriter(join(output,"casos_merged.xlsx"),
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

casos_merged.to_excel(writer,sheet_name='casos_merged',index=None)
worksheet = writer.sheets['casos_merged']
auto_fit_columns(worksheet,casos_merged)

obitos_merged.to_excel(writer,sheet_name='obitos_merged',index=None)
worksheet = writer.sheets['obitos_merged']
auto_fit_columns(worksheet,obitos_merged)

casos_regionais.to_excel(writer,sheet_name='casos_regionais')
worksheet = writer.sheets['casos_regionais']
auto_fit_columns(worksheet,casos_regionais)

casos_municipios.to_excel(writer,sheet_name='casos_municipios')
worksheet = writer.sheets['casos_municipios']
auto_fit_columns(worksheet,casos_municipios)

writer.save()
writer.close()

In [12]:
merge_casos_notifica = pd.merge(merge_casos_notifica, casosn[['id','uf_residencia','classificacao_final','criterio_classificacao','exame','metodo','evolucao','excluir_ficha','manter','duplicado']], on='id', how='left')

merge_casos_notifica.loc[merge_casos_notifica['classificacao_final'].isnull(),'classificacao_final'] = "SEM NOTIFICACAO"
merge_casos_notifica['caso'] = 1
print(len(merge_casos_notifica))

661109


In [13]:
gcf = merge_casos_notifica.groupby('classificacao_final')[['caso','obito','recuperado','ativo']].sum()
gcf = gcf.rename(columns={'caso':'casos','obito':'obitos','recuperado':'recuperados','ativo':'ativos'})
gcf = gcf.append(pd.DataFrame(data=[gcf.sum(axis=0)],columns=gcf.columns,index=['Total']))
gcf

,casos,obitos,recuperados,ativos
0.0,51,0,38,13
1.0,45677,742,26295,18640
2.0,583858,10523,428378,144957
3.0,3941,50,866,3025
5.0,31,0,2,29
SEM NOTIFICACAO,27551,667,16596,10288
Total,661109,11982,472175,176952


In [14]:
gce = merge_casos_notifica.groupby('evolucao')[['caso','obito','recuperado','ativo']].sum()
gce = gce.rename(columns={'caso':'casos','obito':'obitos','recuperado':'recuperados','ativo':'ativos'})
gce = gce.append(pd.DataFrame(data=[gce.sum(axis=0)],columns=gce.columns,index=['Total']))
gce

,casos,obitos,recuperados,ativos
0.0,4967,100,1129,3738
1.0,399169,596,383548,15025
2.0,9901,9682,42,177
3.0,219409,912,70846,147651
4.0,112,25,14,73
Total,633558,11315,455579,166664


In [15]:
merge_casos_notifica = merge_casos_notifica[['ordem', 'identificacao', 'id','uf_residencia','ibge_resid', 'ibge_atend', 'paciente', 'sexo', 'idade', 'mun_resid', 'mun_atend', 'rs', 'classificacao_final', 'laboratorio', 'exame', 'metodo', 'data_diagnostico', 'data_comunicacao', 'data_1o_sintomas','evolucao', 'recuperado', 'caso', 'obito', 'ativo', 'data_obito', 'data_com_obito', 'excluir','excluir_ficha','manter','duplicado']]

In [16]:
obitos_nao_notifica = merge_casos_notifica.loc[(merge_casos_notifica['obito']==1) & (merge_casos_notifica['evolucao']!=2)].copy()
obitos_nao_casos_confirmados = merge_casos_notifica.loc[(merge_casos_notifica['obito']==0) & (merge_casos_notifica['evolucao']==2)].copy()
recuperados_nao_notifica = merge_casos_notifica.loc[(merge_casos_notifica['recuperado']==1) & (merge_casos_notifica['evolucao']!=1)].copy()
recuperados_nao_casos_confirmados = merge_casos_notifica.loc[(merge_casos_notifica['recuperado']==0) & (merge_casos_notifica['evolucao']==1)].copy()

obitos_nao_notifica['diff'] = 'obitos_nao_notifica'
obitos_nao_casos_confirmados['diff'] = 'obitos_nao_casos_confirmados'
recuperados_nao_notifica['diff'] = 'recuperados_nao_notifica'
recuperados_nao_casos_confirmados['diff'] = 'recuperados_nao_casos_confirmados'

diferencas = obitos_nao_notifica.append(obitos_nao_casos_confirmados.append(recuperados_nao_notifica.append(recuperados_nao_casos_confirmados)))

gdiff = diferencas.groupby('diff')[['id']].count() #'caso','obito','recuperado','ativo'
# gdiff = gdiff.rename(columns={'caso':'casos','obito':'obitos','recuperado':'recuperados','ativo':'ativos'})
gdiff = gdiff.append(pd.DataFrame(data=[gdiff.sum(axis=0)],columns=gdiff.columns,index=['Total']))
gdiff

,id
obitos_nao_casos_confirmados,219
obitos_nao_notifica,1633
recuperados_nao_casos_confirmados,15621
recuperados_nao_notifica,72031
Total,89504


In [17]:
casos_comunicados = merge_casos_notifica

writer = pd.ExcelWriter(join(output,"casos_comunicados.xlsx"),
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')


casos_comunicados.to_excel(writer,sheet_name='casos_comunicados',index=None)
worksheet = writer.sheets["casos_comunicados"]
auto_fit_columns(worksheet,casos_comunicados)

writer.save()
writer.close()

In [18]:
recuperados = merge_casos_notifica.loc[(merge_casos_notifica['evolucao']==1) & (merge_casos_notifica['obito']!=1)].copy()
recuperados = pd.merge(recuperados,casosn[['id','data_cura_obito','origem']], how='left', on='id')

recuperados['recuperado'] = 'SIM'
recuperados['data'] = date.today()
recuperados['fonte'] = recuperados['origem']
recuperados.loc[recuperados['data_cura_obito'].isnull(),'data_cura_obito'] = date.today()

recuperados = recuperados[['ordem','identificacao','id','recuperado','data','fonte','mun_resid','ibge_resid','paciente','sexo','idade','data_cura_obito']]

writer = pd.ExcelWriter(join(output,"recuperados.xlsx"),
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')


recuperados.to_excel(writer,sheet_name='recuperados',index=None)
worksheet = writer.sheets["recuperados"]
auto_fit_columns(worksheet,recuperados)

writer.save()
writer.close()

In [20]:
writer = pd.ExcelWriter(join(output,"merge_casos_notifica_classificacao_final.xlsx"),
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

gcf.to_excel(writer,sheet_name='totais')
worksheet = writer.sheets["totais"]
auto_fit_columns(worksheet,gcf)

for group, df in merge_casos_notifica.groupby('classificacao_final'):
    group = str(group)
    df.to_excel(writer,sheet_name=group,index=None)
    worksheet = writer.sheets[group]
    auto_fit_columns(worksheet,df)

writer.save()
writer.close()

In [21]:
writer = pd.ExcelWriter(join(output,"merge_casos_notifica_evolucao.xlsx"),
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

gce.to_excel(writer,sheet_name='totais')
worksheet = writer.sheets["totais"]
auto_fit_columns(worksheet,gce)

for evolucao, df in merge_casos_notifica.groupby('evolucao'):
    df.to_excel(writer,sheet_name=f"{evolucao}",index=None)
    worksheet = writer.sheets[f"{evolucao}"]
    auto_fit_columns(worksheet,df)

writer.save()
writer.close()

In [22]:
writer = pd.ExcelWriter(join(output,"merge_casos_notifica_diferencas.xlsx"),
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

gdiff.to_excel(writer,sheet_name='totais')
worksheet = writer.sheets["totais"]
auto_fit_columns(worksheet,gdiff)

for diff, df in diferencas.groupby('diff'):
    diff = diff[:30] if len(diff) > 30 else diff
    df.to_excel(writer,sheet_name=f"{diff}",index=None)
    worksheet = writer.sheets[f"{diff}"]
    auto_fit_columns(worksheet,df)

writer.save()
writer.close()